In [ ]:
import numpy as np
import pandas as pd
import collections
import modutils
import re
import pickle

w2v_file = '../DataSets/Quora/w2v_src_180115.pickle'
res_file = '../DataSets/Quora/train.pickle'

In [2]:
%%time
src = pd.read_csv(src_file)

Wall time: 1.16 s


In [3]:
src.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
0,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
1,5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
2,8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
3,11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
4,14,14,29,30,What are the laws to change your status from a...,What are the laws to change your status from a...,0


In [4]:
all_text = list(src.question1) + list(src.question2)

In [5]:
len(all_text)

485012

### Lazy dictionary -- no transformations used 

In [9]:
%%time
all_text0 = [[z.lower() for z in str(x).split(' ')] for x in all_text]

Wall time: 3.79 s


In [10]:
dict0 = collections.Counter([z for x in all_text0 for z in x])

In [17]:
dict0i = list(enumerate(sorted(dict0.items(), key=lambda x:x[1], reverse=True)))

In [23]:
total_freq = sum([x[1][1] for x in dict0i])
cumul_freq = np.cumsum([z[1][1] for z in dict0i])
dict0s = [(x[0], x[1], x[1]/total_freq, cumul_freq[i]/total_freq) for (i,x) in dict0i]

In [73]:
#50% - 72
#75% - 1240 (404 occur.)
#90% - 8050 (41 occur.)
#95% - 21750 (10 occur.)
#98% - 59700 (2 occur.)
dict0s[100]

('life?', 4610, 0.0008591084504218018, 0.52923093207862504)

In [75]:
dict0s[:500]

[('the', 225537, 0.04203052984442124, 0.042030529844421238),
 ('what', 186223, 0.03470406788783063, 0.076734597732251858),
 ('is', 161576, 0.030110912578167687, 0.10684551031041956),
 ('how', 131504, 0.024506767389212283, 0.13135227769963184),
 ('i', 127884, 0.023832152944412516, 0.15518443064404436),
 ('a', 125970, 0.023475464533543246, 0.1786598951775876),
 ('to', 122636, 0.022854148357034292, 0.20151404353462188),
 ('in', 116900, 0.0217852012699151, 0.223299244804537),
 ('of', 94901, 0.01768552083589575, 0.24098476564043272),
 ('do', 92813, 0.017296406205856547, 0.25828117184628929),
 ('are', 87091, 0.01623006812487747, 0.27451123997116678),
 ('and', 79351, 0.014787660444559737, 0.28929890041572648),
 ('can', 68251, 0.012719091290615703, 0.30201799170634219),
 ('for', 61029, 0.011373216837482027, 0.31339120854382424),
 ('you', 50611, 0.009431743554077616, 0.32282295209790185),
 ('why', 45383, 0.008457466118328118, 0.33128041821622994),
 ('my', 42569, 0.00793305588416609, 0.339213474

In [80]:
[x for x in dict0s if '?' in x[0]][:10]
#if last is ? (may be multiple) then change to two tokens
#if word (>75% of symbols are alphas) and last is .,:;! than also split

[('india?', 9653, 0.0017989097336055645, 0.47869336499921356),
 ('quora?', 4991, 0.0009301106889490699, 0.52305858262387006),
 ('why?', 4788, 0.0008922800999174808, 0.52576766278521092),
 ('life?', 4610, 0.0008591084504218018, 0.52923093207862504),
 ('it?', 4104, 0.0007648115142149836, 0.53565244161976044),
 ('do?', 3473, 0.0006472198803286155, 0.55108378115060153),
 ('time?', 2814, 0.0005244102341620282, 0.56681198830867074),
 ('me?', 2357, 0.00043924481944559367, 0.58116431744299513),
 ('work?', 2352, 0.00043831303153841166, 0.58160263047453353),
 ('online?', 2326, 0.00043346773442106524, 0.58247441124049293)]

In [99]:
[x for x in dict0s if "'" in x[0]][:10]
#if last is ? (may be multiple) then change to two tokens
#[x for x in dict0s if '3' in x[0]][:10]

[("what's", 7490, 0.0013958182849586323, 0.49220074885930526),
 ("don't", 4683, 0.0008727125538666589, 0.52664037533907759),
 ("i'm", 3652, 0.0006805778874057311, 0.5470776522224633),
 ("can't", 2231, 0.0004157637641846073, 0.58799301829956907),
 ("doesn't", 1754, 0.00032687119783944476, 0.60831121268841681),
 ("you've", 1293, 0.00024096035279726457, 0.63801605209663459),
 ("it's", 1066, 0.00019865718181120187, 0.65568349624713107),
 ("didn't", 942, 0.00017554884171308834, 0.66854999638466295),
 ("someone's", 825, 0.0001537450046850296, 0.67929910168191443),
 ("isn't", 743, 0.00013846368300724483, 0.690395391153382)]

### Lazy dictionary v1.1 -- qst. marks and punctuation

In [100]:
%%time
all_text1_p0 = [[z.lower() for z in str(x).split(' ')] for x in all_text]

Wall time: 4.1 s


In [232]:
def process_text_tuples(lst):
    tmp = [[x] if type(x) is not tuple else list(x) for x in lst]
    return [z for x in tmp for z in x if len(z) > 0]

In [ ]:
regex_alphas = re.compile('[a-z]')

def test_is_word(wrd, pct=0.8):
    if wrd is None or len(wrd) == 0:
        return False
    nwrd = regex_alphas.sub('', wrd)
    len0 = len(wrd)
    len1 = len0 - len(nwrd)
    return float(len1) > pct*float(len0)

In [183]:
def process_text_last_symbol(wrd, symbol, check_duplicates=True, save_duplicates=False):
    """
    Strips last symbol. If check_duplicates will remove several duplicate symbols. If save_duplicates exact number of symbols will be saved, instead of 1
    """
    if wrd is None or len(wrd) == 0 or symbol not in wrd:
        return wrd
    
    if wrd.replace(symbol,'') == '':
        return wrd
    
    k = 1
    while k < len(wrd) and wrd[-k]==symbol:
        k += 1
        if not check_duplicates:
            break
    k -= 1
    if k == 0:
        return wrd
    
    return (wrd[:-k], symbol * (k if save_duplicates else 1))

def process_text_mid_symbol(wrd, symbol, save_duplicates=False, pct_word=0.8):
    """
    Splits text by symbol. Removes empty strings. If save_duplicates then saves multiple symbols (e.g. '..').
    Param pct_words defines what is word -- percent of a-z symbols in result.
    Front and back occurancies should already be handled -- they will be dissmissed otherwise 
    """
    if wrd is None or len(wrd) == 0 or symbol not in wrd:
        return wrd
    
    if wrd.replace(symbol,'') == '':
        return wrd
    
    tmp = wrd.split(symbol)
    tmp0 = []
    tmp1 = []
    for x in tmp:
        if len(x) > 0:
            tmp0.append(x)
            tmp1.append(1)
        else:
            if len(tmp1) == 0:
                continue
            tmp1[-1] += 1
            
    if not all([test_is_word(x) for x in tmp0]):
        return wrd
    tmp = list(zip(*(tmp0, [symbol * (y if save_duplicates else 1) for y in tmp1])))
    tmp = [y for x in tmp for y in x]
    return tuple(tmp[:-1])

In [184]:
%%time
all_text1_p1 = [process_text_tuples([process_text_last_symbol(z, '?', check_duplicates=True) for z in x]) for x in all_text1_p0]
all_text1_p2 = [process_text_tuples([process_text_last_symbol(z, '!', check_duplicates=True) for z in x]) for x in all_text1_p1]
all_text1_p3 = [process_text_tuples([process_text_last_symbol(z, '.', check_duplicates=True) for z in x]) for x in all_text1_p2]
all_text1_p4 = [process_text_tuples([process_text_last_symbol(z, ',', check_duplicates=True) for z in x]) for x in all_text1_p3]
all_text1_p5 = [process_text_tuples([process_text_last_symbol(z, ':', check_duplicates=True) for z in x]) for x in all_text1_p4]
all_text1_p6 = [process_text_tuples([process_text_last_symbol(z, ';', check_duplicates=True) for z in x]) for x in all_text1_p5]

Wall time: 52.7 s


In [233]:
%%time
all_text1_p7 = [process_text_tuples([process_text_mid_symbol(z, ',') for z in x]) for x in all_text1_p6]
all_text1_p8 = [process_text_tuples([process_text_mid_symbol(z, '.', save_duplicates=True) for z in x]) for x in all_text1_p7]
all_text1_p9 = [process_text_tuples([process_text_mid_symbol(z, '-', save_duplicates=True) for z in x]) for x in all_text1_p8]
all_text1_p10 = [process_text_tuples([process_text_mid_symbol(z, '?') for z in x]) for x in all_text1_p9]
all_text1_p11 = [process_text_tuples([process_text_mid_symbol(z, '!') for z in x]) for x in all_text1_p10]
all_text1_p12 = [process_text_tuples([process_text_mid_symbol(z, ':') for z in x]) for x in all_text1_p11]
all_text1 = [process_text_tuples([process_text_mid_symbol(z, ';') for z in x]) for x in all_text1_p12]

Wall time: 1min 12s


In [234]:
dict1 = collections.Counter([z for x in all_text1 for z in x])
dict1i = list(enumerate(sorted(dict1.items(), key=lambda x:x[1], reverse=True)))

total_freq = sum([x[1][1] for x in dict1i])
cumul_freq = np.cumsum([z[1][1] for z in dict1i])
dict1s = [(x[0], x[1], x[1]/total_freq, cumul_freq[i]/total_freq) for (i,x) in dict1i]

In [235]:
#50% - 43 (was 72)
#75% - 620 (was 1240) (915 occur.)
#90% - 3650 (was 8050) (112 occur.)
#95% - 9100 (was 21750) (27 occur.)
#98% - 24900 (was 59700) (5 occur.)
dict1s[1000]

('needing', 555, 9.224531069716512e-05, 0.79464498519587423)

In [244]:
fin_dict = {x[0]:i for (i,x) in enumerate(dict1s)}

In [246]:
sentence_collection = [[fin_dict[z] for z in x] for x in all_text1]

In [251]:
w2v_src = (dict1s, sentence_collection)

In [256]:
with open(w2v_file, 'wb') as f:
    pickle.dump(w2v_src, f)

In [309]:
[(x,i) for (i,x) in enumerate(dict1s) if x[0]=='3-4']

[(('3-4', 50, 8.310388351095958e-06, 0.93258446761823932), 6236)]